# OpenAI Parameters

# Overview
When making requests to OpenAI models, several parameters can be used to control the behavior and output of the model. \
Understanding these parameters helps in fine-tuning the responses to meet specific requirements, whether for generating text, answering questions, or any other use case.

For more detailed examples, refer to the official documentation [Azure OpenAI Service](https://learn.microsoft.com/en-us/azure/ai-services/openai/reference)


In [0]:
pip install openai

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
pip install tiktoken

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
pip install python-dotenv

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
import re
import requests
import sys
import os
from openai import AzureOpenAI
import tiktoken
from dotenv import load_dotenv
load_dotenv()

Out[3]: False

In [0]:
os.environ["AZURE_OPENAI_ENDPOINT"] 
os.environ["AZURE_OPENAI_API_KEY"] 

In [0]:
import re
import requests
import sys
import os
from openai import AzureOpenAI
import tiktoken
from dotenv import load_dotenv
load_dotenv()


client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"),
  api_key=os.getenv("AZURE_OPENAI_API_KEY"),
  api_version="2024-05-01-preview"
)

# Parameter: max_tokens
**Description**: The maximum number of tokens to generate in the completion. \
**Default**: 16 \
**Example**: max_tokens=50

The token count of your prompt plus max_tokens can't exceed the model's context length. \
Most models have a context length of 2048 tokens (except for the newest models, which support 4096). Please refer to documentation.

In [0]:
def call_openai_with_max_tokens(max_tokens):
    response = client.chat.completions.create(
          model="gpt-4o-mini",
          messages = [{"role":"system", "content":"You are a helpful assistant."},
                    {"role":"user","content": "The best pet is a"}],
                    max_tokens=max_tokens
    )
    return client

# Generate with different presence_penalty values
penalties = [16, 32, 60, 100]
for penalty in penalties:
    print(f"Max Tokens: {penalty}\n")
    print(call_openai_with_max_tokens(penalty))
    print("\n" + "-"*80 + "\n")

Max Tokens: 16


--------------------------------------------------------------------------------

Max Tokens: 32


--------------------------------------------------------------------------------

Max Tokens: 60


--------------------------------------------------------------------------------

Max Tokens: 100


--------------------------------------------------------------------------------



# Parameter: temperature

**Description**: Controls the randomness of the output. Lower values make the output more deterministic, while higher values increase randomness. \
**Value Range**: 0 to 1 \
**Default Value**: 1 \
**Example**: temperature=0.7

Higher values means the model will take more risks. \
Try 0.9 for more creative applications, and 0 (argmax sampling) for ones with a well-defined answer.

---
**NOTE**: We generally recommend altering this or top_p but not both.


In [0]:
# Inicializar cliente para Azure OpenAI
def call_openai(num_times, prompt, temperature=0.75):
    for i in range(num_times):
        try:
            # Crear la solicitud a la API
            response = client.chat.completions.create(
                model="gpt-4o-mini",  # Cambia al modelo deseado
                messages=[{"role": "user", "content": prompt}],
                max_tokens=60,
                temperature=temperature
            )
            # Imprimir el contenido de la respuesta
            print(f"Response {i+1}: {response.choices[0].message.content}")
        except Exception as e:
            print(f"An error occurred during request {i+1}: {e}")


In [0]:
# Without seed and temperature, the response is different each time
call_openai(10, 'The best pet is a ')

Response 1: The best pet can vary greatly depending on individual preferences, lifestyles, and living situations. Some popular options include:

1. **Dogs** - Known for their loyalty and companionship, dogs can be great for active individuals or families.
2. **Cats** - Often more independent, cats can be perfect for
Response 2: The best pet can vary depending on individual preferences, lifestyles, and living situations. Here are a few popular options:

1. **Dogs**: Known for their loyalty and companionship, dogs are often considered "man's best friend." They require regular exercise and social interaction.

2. **Cats**: Independent
Response 3: The best pet can vary greatly depending on individual preferences, lifestyles, and living situations. Some popular options include:

- **Dogs**: Known for their loyalty and companionship, they require regular exercise and social interaction.
- **Cats**: Independent yet affectionate, they often require less attention compared to dogs and
Response 

In [0]:
# Now using a seed and 0 temperature, the response is the much more consisitent
call_openai(10, 'The best pet is a ', temperature = 0)

Response 1: The best pet can vary greatly depending on individual preferences, lifestyles, and living situations. Some people might consider dogs the best pets due to their loyalty and companionship, while others might prefer cats for their independence and low maintenance. Small animals like hamsters, rabbits, or guinea pigs can be great for those
Response 2: The best pet can vary greatly depending on individual preferences, lifestyles, and living situations. Some people might consider dogs the best pets due to their loyalty and companionship, while others might prefer cats for their independence and low maintenance. Small animals like hamsters, rabbits, or guinea pigs can be great for those
Response 3: The best pet can vary greatly depending on individual preferences, lifestyles, and living situations. Some people might consider dogs the best pets due to their loyalty and companionship, while others might prefer cats for their independence and low maintenance. Small animals like hams

# Parameter: n
**Description**: Specifies the number of completions to generate for each prompt. \
**Default Value**: 1 \
**Example**: n = 3 

---
**Note**: Because this parameter generates many completions, it can quickly consume your token quota. Use carefully and ensure that you have reasonable settings for max_tokens and stop.

In [0]:
import re
import requests
import sys
import os
from openai import AzureOpenAI
import tiktoken
from dotenv import load_dotenv
load_dotenv()

response = client.chat.completions.create(
          model="gpt-4o-mini",
          messages = [{"role":"system", "content":"You are a helpful assistant."},
                    {"role":"user","content": "The best pet is a"}],
                    max_tokens=40
    )
for index, c in enumerate(response.choices):
    print(index, c.message.content)

0 The best pet is subjective and depends on individual preferences, lifestyles, and circumstances. Some people might say dogs are the best pets because they are loyal and social companions. Others might prefer cats for their independence


# Parameter: presence_penalty
**Description**: Penalizes new tokens based on whether they appear in the text so far, encouraging the model to use new tokens. \
**Value Range**: -2.0 to 2.0 \
**Default Value**: 0 \
**Example**: presence_penalty=0.5

In [0]:
import os
import openai

# Configura tu clave de API de OpenAI
openai.api_key = os.getenv('OPENAI_API_KEY')

def call_openai_with_presence_penalty(presence_penalty):
    response = client.chat.completions.create(
        model="gpt-4o-mini",  # Cambia el modelo según tu necesidad
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": "The best pet is a"}
        ],
        max_tokens=40,
        presence_penalty=presence_penalty  # Aplicamos el parámetro correctamente
    )
    return response.choices[0].message.content  # Devuelve solo el contenido generado

# Generar respuestas con diferentes valores de presence_penalty
penalties = [0, 0.5, 1.0, 1.5, 2.0]
for penalty in penalties:
    print(f"Presence Penalty: {penalty}\n")
    print(call_openai_with_presence_penalty(penalty))
    print("\n" + "-"*80 + "\n")


Presence Penalty: 0

The best pet often depends on individual preferences, lifestyles, and living situations. Some people might say a dog is the best pet because they are loyal, affectionate, and great companions. Others may prefer cats

--------------------------------------------------------------------------------

Presence Penalty: 0.5

The best pet often depends on a person's lifestyle, preferences, and living situation. Commonly favored pets include:

1. **Dogs**: Loyal companions known for their intelligence and playfulness. They require

--------------------------------------------------------------------------------

Presence Penalty: 1.0

The best pet often depends on individual preferences, lifestyle, and living situation. Here are a few popular options and what makes them great:

1. **Dogs**: Known for their loyalty and companionship.

--------------------------------------------------------------------------------

Presence Penalty: 1.5

The best pet can vary greatly depen

# Parameter: frequency_penalty
**Description**: Penalizes new tokens based on their existing frequency in the text so far, reducing the likelihood of repeating the same line verbatim. \
**Value Range**: -2.0 to 2.0 \
**Default Value**: 0 \
**Example**: frequency_penalty=0.5

#### Use cases to explore
1. **Compare Responses** \
Generate multiple completions to compare and choose the best response for your use case.

2. **Increase Diversity** \
Use multiple completions to get a variety of responses, which is useful in creative applications.

3. **Enhance Robustness** \
Generate multiple responses to ensure consistency and accuracy across different completions.

#### Best Practices
1. **Optimize Prompt Length** \
Keep your prompts concise but informative to ensure the model has enough context.

2. **Adjust Temperature and Top_p** \
Use these parameters to balance between deterministic and creative responses.

3. **Monitor Token Usage** \
Be mindful of the max_tokens parameter to manage costs and response length.

4. **Use Stopping Sequences** \
Define stopping sequences to control where the model should stop generating text, ensuring the output is within the desired context.

5. **Generate Multiple Completions** \
Use the n parameter to generate multiple completions and select the best one for your needs.

## Exercise 1: Exploring 'top_p' Parameter
### Description:
- The 'top_p' parameter controls nucleus sampling. Lower values make the output more focused by limiting the pool of tokens.

### Task:
- Write a function that calls the model with different 'top_p' values and observe how the output changes.


In [0]:
def call_openai_with_top_p(top_p):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": "The best pet is a"}
        ],
                    max_tokens=60,
                    top_p= top_p
    )
    return response.choices[0].message.content

# Try with different values of top_p
top_p_values = [0.1, 0.3, 0.7, 1.0]
for value in top_p_values:
    print(f"Top P: {value}\n")
    print(call_openai_with_top_p(value))
    print("\n" + "-"*80 + "\n")


Top P: 0.1

The best pet can vary greatly depending on individual preferences, lifestyles, and living situations. Here are a few popular options:

1. **Dogs**: Known for their loyalty and companionship, dogs are often considered "man's best friend." They require regular exercise and social interaction.

2. **Cats**:

--------------------------------------------------------------------------------

Top P: 0.3

The best pet can vary greatly depending on individual preferences, lifestyles, and living situations. Here are a few popular options:

1. **Dogs**: Known for their loyalty and companionship, dogs are often considered "man's best friend." They require regular exercise and social interaction.

2. **Cats**:

--------------------------------------------------------------------------------

Top P: 0.7

The best pet often depends on individual preferences, lifestyle, and living situation. Here are a few popular options:

1. **Dogs**: Known for their loyalty and companionship, dogs are g

## Exercise 2: Frequency Penalty Exploration
### Description:
- This exercise will explore how the 'frequency_penalty' parameter affects the model's tendency to repeat itself.

### Task:
- Generate completions using different frequency_penalty values and note the diversity in responses.



In [0]:
def call_openai_with_frequency_penalty(frequency_penalty):
    response = client.chat.completions.create(
            model="gpt-4o-mini",
          messages = [{"role":"system", "content":"You are a helpful assistant."},
                    {"role":"user","content": "Tell me a fun fact about space."}],
                    max_tokens=60,
                    frequency_penalty=frequency_penalty
    )
    return response.choices[0].message.content

penalties = [-1.0, 0.0, 1.0, 2.0]
for penalty in penalties:
    print(f"Frequency Penalty: {penalty}\n")
    print(call_openai_with_frequency_penalty(penalty))
    print("\n" + "-"*80 + "\n")


Frequency Penalty: -1.0

A fun fact about space is that it is completely silent! Unlike on Earth, where sound travels through air, space is a vacuum, meaning there are no molecules to carry sound waves. This is why astronauts use radios to communicate while in space, as radio waves can travel through the vacuum, but sound

--------------------------------------------------------------------------------

Frequency Penalty: 0.0

A fun fact about space is that a day on Venus is longer than a year on Venus! Venus has an extremely slow rotation on its axis, taking about 243 Earth days to complete one full rotation. In contrast, it takes only about 225 Earth days for Venus to orbit the Sun. This

--------------------------------------------------------------------------------

Frequency Penalty: 1.0

A fun fact about space is that a day on Venus is longer than a year on Venus! It takes Venus about 243 Earth days to complete one rotation on its axis, but only about 225 Earth days to orbit the

## Exercise 3: Multi-Completion with 'n' Parameter
### Description:
- This exercise will help students generate multiple completions for the same prompt and compare the results.

### Task:
- Generate 3 completions for the same prompt by adjusting the 'n' parameter.


In [0]:
response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages = [{"role":"system", "content":"You are a helpful assistant."},
                    {"role":"user","content": "Tell me a fun fact about space."}],
        n=3
    )
for index, c in enumerate(response.choices):
    print(index, c.message.content)

0 A fun fact about space is that it is completely silent. Unlike on Earth, where sound travels through air, space is a vacuum, meaning there are no air molecules to transmit sound waves. This is why astronauts use radio communication to talk to each other while in space!
1 Sure! One fun fact about space is that there are more stars in the universe than there are grains of sand on all the beaches on Earth. It's estimated that there are around 100 billion to 200 billion galaxies in the observable universe, each containing millions or even billions of stars. This means that the total number of stars could be around 1 septillion (1,000,000,000,000,000,000,000,000)! That puts the vastness of space into perspective and highlights just how enormous the universe is!
2 A fun fact about space is that a day on Venus is longer than a year on Venus! Venus takes about 243 Earth days to rotate once on its axis (a single day) but only about 225 Earth days to complete one orbit around the Sun (a year).

## Exercise 4: Temperature vs. Deterministic Output
### Description:
- Adjust the 'temperature' parameter to explore the balance between creative and deterministic responses.

### Task:
- Generate completions with different temperature values and analyze the variance in output.


In [0]:
def call_openai_with_temperature(temperature):
    response = client.chat.completions.create(
            model="gpt-4o-mini",
          messages = [{"role":"system", "content":"You are a helpful assistant."},
                    {"role":"user","content": "Tell me a fun fact about space."}],
                    max_tokens=60,
                    temperature= temperature
    )
    return response.choices[0].message.content

temperatures = [0.0, 0.3, 0.7, 1.0]
for temp in temperatures:
    print(f"Temperature: {temp}\n")
    print(call_openai_with_temperature(temp))
    print("\n" + "-"*80 + "\n")

Temperature: 0.0

A fun fact about space is that there are more stars in the universe than there are grains of sand on all the beaches on Earth! It's estimated that there are about 100 billion to 200 billion galaxies, each containing millions or even billions of stars. This means the total number of stars could be

--------------------------------------------------------------------------------

Temperature: 0.3

A fun fact about space is that there are more stars in the universe than grains of sand on all the Earth's beaches! It's estimated that there are about 100 billion to 200 billion galaxies, each containing millions or even billions of stars. This means the total number of stars could be around 1

--------------------------------------------------------------------------------

Temperature: 0.7

A fun fact about space is that there are more stars in the universe than there are grains of sand on all the Earth's beaches! Astronomers estimate that there are about 100 billion to 200